# Day 14: Restroom Redoubt

In [55]:
import re
import time
import numpy as np
from operator import mul
from functools import reduce
import matplotlib.pyplot as plt
from IPython.display import clear_output, display

with open('data/2024-14-example.txt', 'r') as f:
    test_data = f.readlines()
with open('data/2024-14.txt', 'r') as f:
    data = f.readlines()

In [89]:
def format_input(str_data: list):
    robot_positions = np.zeros((len(str_data), 2))
    robot_velocities = np.zeros((len(str_data), 2))

    for i, line in enumerate(str_data):
        p = re.search(r'p=(-?\d+),(-?\d+)', line)
        v = re.search(r'v=(-?\d+),(-?\d+)', line)
        
        robot_positions[i, 0] = int(p.group(2))
        robot_positions[i, 1] = int(p.group(1))
        robot_velocities[i, 0] = int(v.group(2))
        robot_velocities[i, 1] = int(v.group(1))
    robot_positions = np.array(robot_positions).astype(int)
    robot_velocities = np.array(robot_velocities).astype(int)
    return robot_positions, robot_velocities


def get_robot_positions(start_pos: np.ndarray, start_vel: np.ndarray, time: int, tile_size: tuple):
    return (start_pos + start_vel * time) % np.array(tile_size)


def get_tile_population(positions: np.ndarray, tile_size: tuple):
    tiles = np.zeros(tile_size, dtype=int)
    for pos in positions:
        tiles[*pos] += 1
    
    return tiles


def quadrant_sum_prod(tiles: np.ndarray):
    # remove the center row and columns
    tiles = np.delete(np.delete(tiles, tiles.shape[0]//2, axis=0), tiles.shape[1]//2, axis=1)
    
    # sum each quadrant
    quadrants_sum = [q.sum().item() for sub in np.split(tiles, 2, axis=0) for q in np.split(sub, 2, axis=1)]
    
    # return the product of the sums
    return reduce(mul, quadrants_sum)


def contains_subarray(arr, sub_arr):
    n = len(arr)
    m = len(sub_arr)

    # Iterate over all possible starting indices
    for i in range(n - m + 1):
        isSame = True
        for j in range(m):
          
            # If any character does not match, break
            # and begin from the next starting index
            if arr[i + j] != sub_arr[j]:
                isSame = False
                break
        
        # If all characters are matched, store the starting index
        if isSame:
            return True
    return False

## Part 1

In [ ]:
time = 100
width = 101
height = 103
tile_size = (height, width)
robot_positions, robot_velocities = format_input(data)
pos_t = get_robot_positions(robot_positions, robot_velocities, time, tile_size)
tiles = get_tile_population(pos_t, tile_size)
quadrant_product = quadrant_sum_prod(tiles)
print('Part 1:', quadrant_product)

## Part 2

In [ ]:
fig, ax = plt.subplots(figsize=(8, 8))
robot_positions, robot_velocities = format_input(data)
pos_t = get_robot_positions(robot_positions, robot_velocities, 0, tile_size)
tiles = get_tile_population(pos_t, tile_size)
mat = ax.matshow(tiles, cmap=plt.cm.Blues)
plt.axis('off')

search_arr = np.ones(10)

found_ts = []
for t in range(0, 10000):
    width = 101
    height = 103
    tile_size = (height, width)
    pos_t = get_robot_positions(robot_positions, robot_velocities, t, tile_size)
    tiles = get_tile_population(pos_t, tile_size)
    
    # set all values to 1 for easier subarray matching
    tiles[tiles > 0] = 1
    
    # assume just 1 axis needs to have a subarray of atleast search_arr
    for i in range(tiles.shape[0]):
        if contains_subarray(tiles[i], search_arr):
            found_ts.append(t)
            print('Found at:', t)
            break
    
    # if t not in found_ts:
    #     for i in range(tiles.shape[1]):
    #         if contains_subarray(tiles[:, i], search_arr):
    #             print('Found at:', t)
    #             found_ts.append(t)
    #             break
    
    if t in found_ts:
        mat.set_data(tiles)
        clear_output(wait=True)
        plt.title(f'Time: {t}')
        display(fig)
        time.sleep(1)
        break
plt.close()